In [1]:
import pulp

## MAKE-TO-STOCK PRODUCTION

Dataset (can be read from a spreadsheet):

In [2]:
# two products
products = ["standard paper", "deluxe paper"]
# price of standard paper and price of deluxe paper (HUF per square meter)
prices = [3000, 4000]
# resources available: wood (in cubic meter) and labor (in hours)
constraints = [40, 100]
# rows are resources, columns are products
# e.g. 0.5 cubic meter of wood is needed for 1 square meter of standard paper
coefficients = [[0.5, 0.5],
                [1.0, 2.0]]

Variables:

In [3]:
xs = [
    pulp.LpVariable("standard paper produced", lowBound=0, cat="Continuous"),
    pulp.LpVariable("deluxe paper produced", lowBound=0, cat="Continuous")
] # non-negative continuous variables

Defining the problem:

In [4]:
problem = pulp.LpProblem("paper production", pulp.LpMaximize) # linear programming - maximization

/opt/conda/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


First, we add the objective function to the problem:

In [5]:
problem += pulp.lpSum([price * x for price, x in zip(prices, xs)]) # objective function to be maximized

Then we add all the constraints (we have already added the nonnegativity constraint earlier, when we created the variables):

In [6]:
for coefficient_row, constraint in zip(coefficients, constraints):
  problem += pulp.lpSum([coefficient * x for coefficient, x in zip(coefficient_row, xs)]) <= constraint

So the problem is the following:

In [7]:
print(problem)

paper_production:
MAXIMIZE
4000*deluxe_paper_produced + 3000*standard_paper_produced + 0
SUBJECT TO
_C1: 0.5 deluxe_paper_produced + 0.5 standard_paper_produced <= 40

_C2: 2 deluxe_paper_produced + standard_paper_produced <= 100

VARIABLES
deluxe_paper_produced Continuous
standard_paper_produced Continuous



Solving the problem:

In [8]:
problem.solve()
print("State:", pulp.LpStatus[problem.status])
for v in problem.variables():
  print(v.name, "=", v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6af183507c5b44918566e0ecb7d453ae-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/6af183507c5b44918566e0ecb7d453ae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 14 RHS
At line 17 BOUNDS
At line 18 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj -0 Dual inf 7000 (2)
0  Obj -0 Dual inf 7000 (2)
2  Obj 260000
Optimal - objective value 260000
Optimal objective 260000 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

State: Optimal
deluxe_paper_produced = 20.0
standard_paper_produced = 60.0


In [9]:
print("Maximal profit: %d HUF" % pulp.value(problem.objective))

Maximal profit: 260000 HUF
